### Pytorch 모델 파일을 tensorflow 파일로 변환, 이미지 입력 후 prediction하고, 후처리하여 출력

아래 코드를 실행시켜 학습이 완료된 pytorch 모델파일(best.pt)을 tensorflow 모델파일(saved model)로 변환합니다.

>{가상환경 경로명}\Scripts\python.exe {yolov5-master 경로명}\export.py --weights {yolov5-master 경로명}\runs\train\exp\weights\best.pt --include saved_model

{yolov5-master 경로명}\runs\train\exp\weights 폴더로 들어가면 tensorflow 모델로 변환된 'saved_model' 폴더가 생성된 것을 확인할 수 있습니다.

'saved_model' 폴더를 실습파일이 있는 경로(현재 보고 있는 이 주피터 노트북 파일이 있는 경로)로 복사해 옵니다.

In [1]:
import cv2
import numpy as np
import tensorflow as tf

In [2]:
#tf saved_model로 변환된 모델 파일 로드
model = tf.saved_model.load("best_saved_model")

In [3]:
#모델의 입출력 구조 확인 → 입력은 (1, 640, 640, 3), 출력은 (1, 25200, 7)임을 알 수 있음
for k, v in model.signatures.items():
    print(k, v)

serving_default ConcreteFunction signature_wrapper(*, x)
  Args:
    x: float32 Tensor, shape=(1, 640, 640, 3)
  Returns:
    {'output_0': <1>}
      <1>: float32 Tensor, shape=(1, 25200, 7)


이미지 하나를 로드해 모델에 입력으로 넣고 prediction한 결과를 봅니다.

In [4]:
#테스트할 이미지를 행렬로 로드
src = cv2.imread("test_images/test_image1.jpg")
src_converted = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
src_converted.shape # -> (2268, 4032, 3) 4032 x 2268 사이즈의 3채널 컬러이미지

(2268, 4032, 3)

In [5]:
#우리가 만든 모델은 (1, 640, 640, 3)을 입력으로 받으므로, 그에 맞게 resize & reshape을 진행
# ※입력에 차원이 하나 더 붙어있는 이유는 배치 때문으로, resize 후 reshape을 통해 변환

src_resized = cv2.resize(src_converted, (640, 640))
src_resized.shape # -> (640, 640, 3)

(640, 640, 3)

In [6]:
src_resized = src_resized.reshape((1, 640, 640, 3))
src_resized.shape

(1, 640, 640, 3)

In [7]:
#추론 실행
pred = model(src_resized)
pred[0].shape # -> TensorShape([1, 25200, 7])  *numpy가 아니라 tf 프레임웍 포맷에 맞춰 출력

TensorShape([1, 25200, 7])

In [8]:
#제일 앞에 있는 1차원은 의미 없고 나머지 차원이 중요 -> 25200행 * 7열
#25200행 : 입력에 따라 추론한 25200개의 객체검출 결과
#7열 : 행별 결과내용 → [cx, cy, w, h, obj conf, c1 conf, c2 conf]
# 0~3번째 열은 검출한 바운딩 박스 정보
# 4번째 열은 바운딩 박스 안에 객체가 있을 확률
# 5번째 열은 검출한 객체가 1번 클래스(도로표지판)일 확률
# 6번째 열은 검출한 객체가 2번 클래스(신호등)일 확률

pred = pred[0][0].numpy() #필요없는 차원들은 제거하고 (25200, 7)만 선택해 numpy 행렬로 변환한 후 pred에 덮어 씌움
pred

array([[ 1.86721645e-02, -1.87345489e-03,  3.46859507e-02, ...,
         2.33685728e-02,  2.97064800e-02,  9.45612609e-01],
       [ 1.86640378e-02,  5.39534260e-03,  2.74656694e-02, ...,
         1.21878463e-09,  1.17530655e-02,  9.89085913e-01],
       [ 1.86193492e-02, -6.50437956e-04,  4.78976294e-02, ...,
         9.69655514e-01,  1.56350777e-01,  7.28169084e-01],
       ...,
       [ 9.94192719e-01,  9.82448399e-01,  5.02370223e-02, ...,
         7.26965227e-05,  6.62724301e-02,  8.54448140e-01],
       [ 9.76914823e-01,  9.73752201e-01,  1.22365519e-01, ...,
         1.79213268e-04,  1.56533509e-01,  8.33407998e-01],
       [ 9.79755223e-01,  9.77258623e-01,  1.05926394e-01, ...,
         8.61017397e-05,  8.64517868e-01,  6.72855452e-02]], dtype=float32)

In [9]:
#pred의 4번째 행(pred[:, 4])를 기준으로 정렬하면 객체가 있을 확률이 높은 순서대로 결과를 볼 수 있음
rank = np.argsort(-pred[:, 4])
pred = pred[rank]
pred

array([[5.82638681e-01, 4.39488262e-01, 3.28190254e-05, ...,
        1.00000000e+00, 1.19516946e-29, 1.00000000e+00],
       [4.32306737e-01, 6.81562662e-01, 1.94896050e-02, ...,
        1.00000000e+00, 1.48033621e-07, 9.99674797e-01],
       [3.43600273e-01, 4.50095862e-01, 5.37785795e-03, ...,
        1.00000000e+00, 9.96729374e-01, 8.34988281e-02],
       ...,
       [7.18276441e-01, 7.68776417e-01, 2.74658203e-02, ...,
        0.00000000e+00, 1.00000000e+00, 1.84853044e-12],
       [6.60836697e-01, 8.81302536e-01, 5.85933179e-02, ...,
        0.00000000e+00, 6.40474451e-10, 9.99373615e-01],
       [6.67989194e-01, 9.06158566e-01, 4.74812929e-03, ...,
        0.00000000e+00, 1.10505754e-03, 9.99997377e-01]], dtype=float32)

추론한 바운딩 박스 정보를 원본 이미지에 그린 후 결과를 봅니다.

In [97]:
def getXyxy(cxcywh):
    x1 = cxcywh[0] - (cxcywh[2]/2)
    y1 = cxcywh[1] - (cxcywh[3]/2)
    x2 = x1 + cxcywh[2]
    y2 = y1 + cxcywh[3]
    return [x1, y1, x2, y2]

def getCandidatesByClass(pred, class_names, conf_thres=0.8):
    class_len = pred[0].shape[0] - 5 #cx, cy, w, h, c 를 뺀 나머지가 클래스의 개수이다.
    candidate_list = []
    for class_no in range(class_len):
        candidate_list.append({
            "class_no" : class_no,
            "class_name" : class_names[class_no],
            "bboxes(xyxy)" : []
        })
    for item in pred:
        this_class_no = item[5:].argmax() #클래스별 확률값을 담고 있는 요소들만 슬라이싱한 후 최대값이 위치한 곳의 index 번호 가져오기
        #################### conf 값 계산이 중요!!!!
        # this_conf = item[4] * item[5 + this_class_no] #obj conf와 class conf를 곱해 최종 conf를 뽑아낸다.
        this_conf = item[4]
        for class_conf in item[5:]:
            this_conf *= class_conf #obj conf와 class conf를 곱해 최종 conf를 뽑아낸다.
        ####################
        if this_conf >= conf_thres:
            this_result = getXyxy(item[:4])
            this_result.append(this_conf)
            candidate_list[this_class_no]["bboxes(xyxy)"].append(this_result)
    for class_no in range(class_len):
        candidate_list[class_no]["bboxes(xyxy)"] = np.array(candidate_list[class_no]["bboxes(xyxy)"])
        candidate_list[class_no]["bboxes(xyxy)"] = candidate_list[class_no]["bboxes(xyxy)"][candidate_list[class_no]["bboxes(xyxy)"][:, -1].argsort()[::-1]]
        candidate_list[class_no]["bbox_cnt"] = candidate_list[class_no]["bboxes(xyxy)"].shape[0]
    return candidate_list

def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

def nms1(candidate_list, iou_thres = 0.45, merge_thres = 0.75, print_iou=False):
    result = candidate_list.copy()
    idx_list_to_delete = []
    for idx, item in enumerate(candidate_list):
        if idx+1 < len(candidate_list):
            next_item = candidate_list[idx+1]
            box1 = item[:4]
            box2 = next_item[:4]
            iou = IoU(item[:4], next_item[:4])
            if print_iou:
                print(iou)
            if iou > merge_thres: #bbox를 합쳐버린다~!
                x1 = min(box1[0], box2[0])
                y1 = min(box1[1], box2[1])
                x2 = max(box1[2], box2[2])
                y2 = max(box1[3], box2[3])
                replace_val = [x1, y1, x2, y2, item[4]]
                result[idx] = replace_val
                idx_list_to_delete.append(idx+1)
            elif iou > iou_thres:
                idx_list_to_delete.append(idx+1)
    result = np.delete(result, idx_list_to_delete, axis=0)
    return result

def nms2(bboxes, iou_thres = 0.45, merge_thres = 0.75, print_iou=False): #겹치는 면적이 iou_thres를 넘으면 하나만 선택한다.
    B = bboxes

    #1) 해당 클래스에 대해 최종적으로 남길 bounding box 를 담을 리스트 D를 생성
    # B 중에서 class score 가 가장 높은 bounding box 를 선택하고 D에 추가하고 B에서 삭제
    argmax_B = B[:, -1].argmax()
    T = B[argmax_B] #B의 최대값을 담을 임시 변수 T 생성
    B = np.delete(B, argmax_B, axis=0)
    D = None #검사가 완료된 최종 D만 담을 리스트

    #2) D에 추가된 class score 가 가장 높은 bounding box 와 B에 담긴 bounding box 와의 IOU (Intersection Over Union) 을 계산
    while True:
        idx_to_delete_from_B = []
        for idx, B_item in enumerate(B):
            # IOU 가 주어진 임계치 iou_thres보다 크다면 B에서 제거
            iou = IoU(T[:4], B_item[:4])
            if iou > iou_thres:
                idx_to_delete_from_B.append(idx)
        B = np.delete(B, idx_to_delete_from_B, axis=0)
        if D is None:
            D = T.copy()
            D = D.reshape(1, 5)
        else:
            D = np.vstack([D, T])
        print("D : {}".format(len(D)), "B : {}".format(len(B)))

        if len(B) > 0:
            #3) 2번 과정을 수행하고 B에 남은 bounding box 중 가장 큰 class score 를 가진 bounding box 를 D에 추가하고 B에서 제거한 후 2번 과정을 반복
            #B에 bounding box 가 남지 않을 때까지 반복
            argmax_B = B[:, -1].argmax()
            T = B[argmax_B]
            B = np.delete(B, argmax_B, axis=0)
        else:
            break
    return D
    

conf_thres = 0.85
class_names = ["sign", "light"]
candidate_list = getCandidatesByClass(pred, class_names, conf_thres)
candidate_list[0] = candidate_list[0]["bboxes(xyxy)"]
candidate_list[1] = candidate_list[1]["bboxes(xyxy)"]
# for candidate in candidate_list:
#     this_bboxes = filterCandidates(candidate["bboxes(xyxy)"])
# for _ in range(10):
#     candidate_list[1] = nms(candidate_list[1])
# candidate_list[0].shape

In [99]:
class_names = ["sign", "light"]
dst = cv2.resize(src, (1600, int((1600*src.shape[0])/src.shape[1]))) #이미지가 너무 크므로 리사이즈한다.
cnt =0
for bbox in candidate_list[1]:
    class_name = class_names[int(bbox[-1])]
    class_conf = bbox[-2]
    color = []
    for _ in range(3):
        color.append(np.random.randint(0, 255))
    # color = (0, 255, 0)

    # cx = int(bbox[0] * dst.shape[1])
    # cy = int(bbox[1] * dst.shape[0])
    # w = int(bbox[2] * dst.shape[1])
    # h = int(bbox[3] * dst.shape[0])
    # x = int(cx - (w/2))
    # y = int(cy - (h/2))

    x1 = int(bbox[0] * dst.shape[1])
    y1 = int(bbox[1] * dst.shape[0])
    x2 = int(bbox[2] * dst.shape[1])
    y2 = int(bbox[3] * dst.shape[0])

    # if w < 10 or h < 10:
    #     continue

    # cnt += 1
    # if cnt > 2:
    #     break

    dst = cv2.rectangle(dst, (x1, y1), (x2, y2), color, 2)
    dst = cv2.putText(dst, "{}".format(int(bbox[4]*100)), (x1, y1), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()


Lessons

1. 물체감지는 라벨링 작업에 시간이 많이 걸리므로, 이번 실습은 일반적으로 공개된 데이터를 활용해 진행할 수 밖에 없었다.

2. 현업에서 실제 활용할 목적으로 모델을 만들고자 한다면, 수집한 이미지 데이터에 맞는 라벨링 작업이 필수로 요구된다.